# Link prediction

data: http://snap.stanford.edu/pathways/bio-pathways-network.tar.gz

In [1]:
!git clone https://github.com/stellargraph/stellargraph.git -q

In [2]:
import sys
sys.path.append('/content/stellargraph')

In [3]:
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk

from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!tar -xzvf "/content/drive/MyDrive/OTUS_ML_advanced/07_Graphs/data/bio-pathways-network.tar.gz"

./._bio-pathways-network.csv
bio-pathways-network.csv


## 1. Загружаем граф

In [6]:
edges = pd.read_csv('/content/bio-pathways-network.csv')

print(f'edges.shape = {edges.shape}\n')
edges.head(2)

edges.shape = (342353, 2)



,Gene ID 1,Gene ID 2
0,1394,2778
1,6331,17999


## 2. Создаем networkx граф из списка связей

In [7]:
G = nx.from_pandas_edgelist(edges, "Gene ID 1", "Gene ID 2", create_using=nx.Graph())
G.number_of_nodes()

21557

## 3. Разметка данных для link prediction

Собираем тренировочный и тестовый наборы, в них должны быть истинно позитивные и истинно негативные ссылки.

* Истинно позитивные - те, что заведомо присутствуют в исходном графе, истинно позитивные в тестовой выборке мы удалим из графа, чтобы получить хорошую тренировочную выборку;
* Истинно негативные - те, что заведомо отсутствуют в исходном графе. Их мы наберем из матрицы смежности.

### 3.1. Матрица смежности

In [8]:
nodes = np.array(list(G.nodes))
adj_G = nx.to_numpy_array(G, nodelist=nodes)
adj_G.shape

(21557, 21557)

### 3.2. Словарь несвязанных вершин

Для каждого столбца в матрице смежности:    
* для каждой строки в матрице смежности:    
   * если связи между столбцом и строкой нет, то добавить соответствующую пару в словарь
   * остановиться, если количество больше либо равно 100

In [9]:
unconnected = {"G1": [], "G2": []}
for i in tqdm(np.arange(adj_G.shape[0])):
  for j in np.arange(i + 1, adj_G.shape[0]):
    if adj_G[i, j] == 0:
      unconnected["G1"].append(nodes[i])
      unconnected["G2"].append(nodes[j])
      if len(unconnected["G1"]) >= 100:
        break
unconnected = pd.DataFrame(unconnected)

100%|██████████| 21557/21557 [00:00<00:00, 59973.95it/s]


In [10]:
unconnected.shape

(21655, 2)

### 3.3. Граф для обучения модели

Истинно негативные - случайная выборка из несвязанных узлов:

In [11]:
n = unconnected.shape[0]
true_zero = unconnected.iloc[np.random.choice(np.arange(n), n, replace=False)]

Истинно позитивные - случайная выборка из списка связей:

In [12]:
test_size = 100
true_zero_train_ix, true_zero_test_ix = train_test_split(np.arange(true_zero.shape[0]), test_size=test_size)
true_one_train_ix, true_one_test_ix = train_test_split(np.arange(edges.shape[0]), test_size=test_size)

Создаём граф без истинно позитивных, попавших в тестовую выборку:

In [13]:
G_no_one = nx.from_pandas_edgelist(edges.iloc[true_one_train_ix], "Gene ID 1", "Gene ID 2", create_using=nx.Graph())

## 4. Эмбеддинги на графах

Конвертируем граф в формат StellarGraph:

In [14]:
G_no_one_st = StellarGraph.from_networkx(G_no_one)

Пробегаемся случайным блужданием по графу.

In [15]:
rw = BiasedRandomWalk(G_no_one_st)

walks = rw.run(nodes=list(G_no_one_st.nodes()), length=20, n=5, p=0.4)
len(walks)

107780

In [16]:
str_walks = [[str(a) for a in b] for b in walks]

for sw in str_walks[:10]:
    print(sw)

['10902', '54556', '6627', '24148', '11198', '79598', '230967', '5108', '57587', '66489', '9669', '64210', '23385', '682', '9416', '9343', '267010', '23020', '155871', '55299']
['10902', '7465', '10902', '1871', '7528', '4149', '89874', '23528', '147746', '23528', '10562', '64216', '708', '26292', '54726', '7846', '3002', '10728', '54984', '4869']
['10902', '10761', '55578', '57520', '6668', '23390', '84905', '2885', '3315', '10075', '4678', '3020', '7090', '27023', '55157', '668', '81608', '6449', '10952', '6261']
['10902', '347918', '23506', '57685', '23291', '6651', '7307', '4076', '9513', '1660', '6206', '8664', '283385', '150483', '84708', '23582', '3725', '51360', '3725', '2516']
['10902', '4149', '6908', '10370', '4087', '26953', '3308', '11196', '7328', '84959', '3065', '54822', '21853', '54822', '21853', '54962', '8914', '100144435', '8914', '18626']
['57088', '26122', '10933', '5931', '8819', '6872', '1457', '51138', '7534', '10921', '81669', '10544', '2149', '5340', '10549',

Обучаем Word2Vec:

In [17]:
model = Word2Vec(str_walks, vector_size=100)

In [18]:
model.wv[str(nodes[0])]

array([ 0.01679301,  0.1824894 ,  0.16080186,  0.22039254,  0.08176728,
       -0.12929738, -0.06809992,  0.7717444 , -0.368339  , -0.33294696,
       -0.3841941 , -0.13527963, -0.20262808,  0.37163085,  0.08271859,
        0.06962608, -0.5277362 , -0.3074973 , -0.04025541, -0.5814117 ,
        0.4396745 ,  0.2943754 ,  0.34357446,  0.28917012,  0.17560579,
        0.13474537, -0.18014222, -0.26098353, -0.23761447, -0.13788566,
        0.03749861, -0.05933555, -0.31201684, -0.45920298, -0.08447216,
        0.0009327 ,  0.44944993, -0.15713733, -0.04288895, -0.8997407 ,
       -0.25492606, -0.37254724, -0.18402913,  0.48480472,  0.43631974,
        0.32088706, -0.62002057, -0.01084175,  0.34936425,  0.3388544 ,
       -0.23595574, -0.50294477,  0.01568941, -0.7198748 , -0.4174407 ,
       -0.13631912, -0.22105783, -0.78161514, -0.57239366, -0.00161625,
        0.36957917,  0.56790864, -0.06096058,  0.00950764, -0.0748058 ,
        0.5407277 ,  0.24942252,  0.5843835 ,  0.08662704,  0.15

In [19]:
print(model.wv)

KeyedVectors<vector_size=100, 21556 keys>


## 5. Link prediction

Обучим следующую модель:    
* Основа - LGBMClassifier   
* Входные данные - вектор из 200 элементов:
    * 1-100 -- эмбеддинг первого узла
    * 101-200 -- эмбеддинг второго узла
* Выходные данные - есть ссылка (1) и нет ссылки (0)
* Оцениваем метриками для бинарной классификации.

### 5.1. Формируем обучающий и тестовый датасет

#### 5.1.1 Истинно негативный набор ссылок

In [20]:
no_one_nodes = list(G_no_one_st.nodes())

In [21]:
filter_b_0 = [
    (true_zero.iloc[a]["G1"] in no_one_nodes) and (true_zero.iloc[a]["G2"] in no_one_nodes)
    for a in np.arange(true_zero.shape[0])
]
true_zero = true_zero[filter_b_0]

Сначала разбираемся с истинно негативным набором ссылок:   
* собираем векторы для первого гена
* собираем векторы для второго гена
* объединяем их, чтобы получить матрицу размерности (N, 100)

In [22]:
G1_0 = np.stack([model.wv[str(true_zero.iloc[a]["G1"])] for a in np.arange(true_zero.shape[0])])
G2_0 = np.stack([model.wv[str(true_zero.iloc[a]["G2"])] for a in np.arange(true_zero.shape[0])])
G1G2_0 = np.concatenate([G1_0, G2_0], 1)

G1G2_0.shape

(21653, 200)

#### 5.1.2. Истинно позитивный набор ссылок

Теперь разбираемся с истинно позитивным набором ссылок:   
* фильтруем те узлы, которые точно присутствуют в графе без истинно позитивного тестового набора;
* собираем векторы для первого гена
* собираем векторы для второго гена
* объединяем их, чтобы получить матрицу размерности (N, 100)

In [23]:
filter_b_1 = [
    (edges.iloc[a]["Gene ID 1"] in no_one_nodes) and (edges.iloc[a]["Gene ID 2"] in no_one_nodes)
    for a in np.arange(edges.shape[0])
]
true_one = edges[filter_b_1]

In [24]:
G1_1 = np.stack([model.wv[str(true_one.iloc[a]["Gene ID 1"])] for a in np.arange(true_one.shape[0])])
G2_1 = np.stack([model.wv[str(true_one.iloc[a]["Gene ID 2"])] for a in np.arange(true_one.shape[0])])
G1G2_1 = np.concatenate([G1_1, G2_1], 1)

G1G2_1.shape

(342352, 200)

In [25]:
G1G2 = np.concatenate([G1G2_1, G1G2_0], 0)
labels = np.array([1] * len(G1G2_1) + [0] * len(G1G2_0))

train_ix, test_ix = train_test_split(np.arange(G1G2.shape[0]), stratify=labels)

### 5.2. Обучаем LGBMClassifier

In [26]:
lgb = LGBMClassifier()
lgb.fit(G1G2[train_ix], labels[train_ix])

y_hat_p = lgb.predict_proba(G1G2[test_ix])
y_hat = lgb.predict(G1G2[test_ix])

###  5.3. Считаем метрики

In [27]:
accuracy_score(y_pred=y_hat, y_true=labels[test_ix])

0.9658359156941606

In [28]:
roc_auc_score(y_score=y_hat, y_true=labels[test_ix])

0.750374289388805

In [29]:
matthews_corrcoef(y_pred=y_hat, y_true=labels[test_ix])

0.6457429662346784

In [30]:
f1_score(y_pred=y_hat, y_true=labels[test_ix])

0.9820723219485753